In [5]:
import sys
from pathlib import Path
PROJECT_ROOT = Path().cwd().resolve().parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

print("Added:", PROJECT_ROOT)

print("CWD:", Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("sys.path contains project root?:", str(PROJECT_ROOT) in sys.path)

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from src.dataset import DeepPHQDataset, build_vocab, create_balanced_dataloader, split_by_pid

from models.attention_rnn.attention_model import AttentionRNN
import yaml

# Load config
CONFIG_PATH = Path("config.yaml")

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

Added: /Users/sun/Desktop/OMSCS/2025_Fall/Deep Learning/Projects/DeepPHQ
CWD: /Users/sun/Desktop/OMSCS/2025_Fall/Deep Learning/Projects/DeepPHQ/models/attention_rnn
PROJECT_ROOT: /Users/sun/Desktop/OMSCS/2025_Fall/Deep Learning/Projects/DeepPHQ
sys.path contains project root?: True


In [7]:
data_cfg = config["data"]
root = Path(data_cfg["data_root"])

word_df = pd.read_csv(root / data_cfg["word_csv"])
sentence_df = pd.read_csv(root / data_cfg["sentence_csv"])
dialogue_df = pd.read_csv(root / data_cfg["dialogue_csv"])

In [10]:
word_df.head()
sentence_df.head()
dialogue_df.head()

,PID,Text,PHQ_Score
0,300,i like the opportunities um yes um it took a m...,2
1,300,grandparents parents um yeah i mean they've al...,2
2,300,playing games grandparents parents um yeah i m...,2
3,300,um i took up business and administration uh ye...,2
4,300,don't travel a lot i'm pretty local once a yea...,2


In [11]:
for df in [word_df, sentence_df, dialogue_df]:
    all_texts = df["Text"].tolist()
    vocab = build_vocab(all_texts, min_freq=config["vocab"]["min_freq"])

    # 1. split
    train_df, val_df, test_df = split_by_pid(df)

    # 2. create datasets
    train_dataset = DeepPHQDataset(
        data=list(zip(train_df["PID"], train_df["Text"], train_df["PHQ_Score"])),
        vocab=vocab,
        max_length=config["data"]["max_length"]
    )

    val_dataset = DeepPHQDataset(
        data=list(zip(val_df["PID"], val_df["Text"], val_df["PHQ_Score"])),
        vocab=vocab,
        max_length=config["data"]["max_length"]
    )

    test_dataset = DeepPHQDataset(
        data=list(zip(test_df["PID"], test_df["Text"], test_df["PHQ_Score"])),
        vocab=vocab,
        max_length=config["data"]["max_length"]
    )

    # 3. dataloaders
    train_loader = create_balanced_dataloader(
        train_dataset,
        batch_size=config["dataloader"]["batch_size"]
    )

    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # 4. verify shapes
    batch = next(iter(train_loader))
    print(batch["input_ids"].shape)
    print(batch["label"].shape)
    print(batch["pid"].shape)

Vocab size = 6246
torch.Size([32, 512])
torch.Size([32])
torch.Size([32])
Vocab size = 6230
torch.Size([32, 512])
torch.Size([32])
torch.Size([32])
Vocab size = 6170
torch.Size([32, 512])
torch.Size([32])
torch.Size([32])


In [12]:
# ---- Load model config ----
model_cfg = config["model"]

# ---- Auto-select device ----
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)

Using device: mps


In [ ]:
model = AttentionHan(
    vocab_size,
    embed_dim=100,
    hidden_dim=50,
    num_classes=22,
    pad_idx=0,
    max_words=300,
    max_sentences=10
).to(device)